<font size="5">**Algorytmy danych geoprzestrzennych**</font><br>
<font size="4">Skrypty geoprzetwarzania w QGIS</font>

<font size="4">Krzysztof Dyba</font>

Narzędzia przetwarzania (*Processing Toolbox*) w QGIS są kompleksowym systemem do uruchamiania algorytmów analizy geoprzestrzennej. Algorytmy te mogą pochodzić z różnych źródeł:

1. Natywne algorytmy (wbudowane bezpośrednio w QGIS i napisane w C++ dla większej wydajności).
2. Zewnętrzne algorytmy (pochodzące z różnych aplikacji, np. SAGA GIS czy GRASS GIS).
3. **Własne skrypty przetwarzania**.

Skrypty przetwarzania są pisane przy użyciu języka Python i wykorzystują QGIS Python API (PyQGIS) do interakcji z danymi przestrzennymi. Skrypty te muszą przestrzegać określonej struktury, w tym definiować dane wejściowe, wyjściowe i logikę algorytmu. Istnieje kilka możliwościa ich zastosowania:

- bezpośrednie uruchomienie z poziomu interfejsu QGIS,
- wykorzystanie w kreatorze modeli (*model designer*) do tworzenia złożonych przepływów pracy,
- użycie do przetwarzania wsadowego,
- odwołanie się z poziomu wbudowanej konsoli Python.

Aby stworzyć nowy skrypt przetwarzania w QGIS należy:

1. Otworzyć panel algorytmów w zakładce Processing (`CTRL` + `ALT` + `T`).
2. Kliknąć ikonę "Twórz nowy skrypt" (alternatywnie można wykorzystać gotowy szablon).
3. Napisać kod zgodnie z wymaganą strukturą.
4. Zapisać plik z rozszerzeniem `.py` w katalogu ze skryptami w QGIS. Teraz skrypt będzie dostępny w zakładce "Skrypty" w panelu algorytmów.

# Struktura skryptu

Do tworzenia skrytpów przetwarzania można wybrać jeden z dwóch sposobów:

1. Wykorzystanie klasy [QgsProcessingAlgorithm](https://qgis.org/pyqgis/3.34/core/QgsProcessingAlgorithm.html). Szablon dostępny jest [tutaj](https://docs.qgis.org/3.40/en/docs/user_manual/processing/scripts.html#scripts-extend).
2. Wykorzystanie dekoratora `@alg`, który umożliwia modyfikację metody lub klasy bez bezpośredniej zmiany ich kodu (*wrapping*). Jest to sposób uproszczony z brakiem możliwości użycia we wtyczkach. Szablon dostępny jest [tutaj](https://docs.qgis.org/3.40/en/docs/user_manual/processing/scripts.html#scripts-alg).

Przykładowe skrypty znajdziesz w [repozytorium QGIS](https://github.com/qgis/QGIS/tree/master/python/plugins/processing/algs/qgis).

# Wyświetlenie metadanych

Zacznijmy od napisania prostego skryptu używając dekoratora `@alg`, który umożliwi wyświetlenie metadanych warstwy wektorowej. Obligatoryjnie musimy zdefiniować trzy elementy:

1. Dekorator `@alg` służący do definiowania nazwy i lokalizacji algorytmu w panelu algorytmów.
2. Dekorator `@alg.input` służący do definiowania danych wejściowych.
3. Dekorator `@alg.output` służący do definiowania danych wyjściowych.

Dane wyjściowe muszą zostać jawnie zdefiniowane, tzn. należy dokładnie wskazać, co jest wynikiem działania algorymtu. W przeciwnym razie nie będzie można poprawnie użyć skryptu w modelerze. Algorytm może zwracać dane różnego typu, np. łańcuchy tekstu, liczby czy wartości logiczne. Wszystkie typy danych wejściowych i wyjściowych dla algorytmów przetwarzania znajdziesz [tutaj](https://docs.qgis.org/3.40/en/docs/user_manual/processing/parameters.html). Do przetwarzania przydatne będą również metody ewaluacji parametrów opisane [tutaj](https://qgis.org/pyqgis/3.40/core/QgsProcessingParameters.html). Jeśli algorytm posiada zdefiniowane wyjście (warstwa wektorowa, rastrowa, chmura punktów), to automatycznie zostanie dodane do projektu.

Danymi wejściowymi w naszej funkcji będzie warstwa wektorowa wybrana przez użytkownika za pomocą interfejsu, a danymi wyjściowymi łańcuch tekstu opisujące metadane tej warstwy (np. nazwa, źródło, liczba obiektów, typ geometrii, zakres przestrzenny oraz układ przestrzenny) jako słownik.

```python
from qgis.processing import alg

@alg(name = "vector_metadata", label = "Print metadata of vector layer",
     group = "examplescripts" , group_label = "Example scripts")

@alg.input(type = alg.SOURCE, name = "INPUT", label = "Input vector layer")

@alg.output(type = alg.STRING, name = "OUTPUT", label = "Metadata")

def vector_metadata(instance, parameters, context, feedback, inputs):
    """
    Print metadata of vector layer
    """
    
    # QgsVectorLayer
    layer = instance.parameterAsVectorLayer(parameters, "INPUT", context)
    
    name = layer.name()
    data_source = layer.dataProvider().dataSourceUri()
    num_features = layer.featureCount()
    geometry_type = layer.geometryType()
    extent = layer.extent().toString(precision = 2)
    crs = layer.crs().authid()
    
    output = {
        "Layer name": name,
        "Data source": data_source,
        "Number of features": num_features,
        "Geometry type": geometry_type,
        "Extent": extent,
        "CRS": crs
    }
    
    # sprawdź czy użytkownik anulował operację
    if feedback.isCanceled():
        return {}

    return output
```

# Inna operacja

Zobaczmy teraz jak wygląda bardziej zaawansowany skrtpy, który wykona jakieś operacje na geometriach i zapisze wynik na dysku.

# Zadania

19) Zaimplementuj wybrany przez siebie algorytm lub narzędzie z poprzednich zajęć jako skrypt przetwarzania w QGIS.